## JEM207 Data Processing in Python - Midterm
[Daniel Brosko](mailto:89227653@fsv.cuni.cz), April 5th, 2022

#### Just testing

In [1]:
# Just testing
import numpy as np

In [2]:
a = np.array([0, 1, 2, 3, 4])

print(a)

a.shape

len(a)

[0 1 2 3 4]


5

In [1]:
import pandas as pd
import sys

In [2]:
# from dict of Series
d = {'one': pd.Series([1., 2., 3.],     index=['a', 'e', 'c']),
     'two': pd.Series([2., 1., 3., 4.], index=['b', 'a', 'c', 'd'])}

df = pd.DataFrame(d)
df

,one,two
a,1.0,1.0
b,NaN,2.0
c,3.0,3.0
d,NaN,4.0
e,2.0,NaN


## Task 1

## Task 2

## Task 3